In [23]:
#Esse script busca de um arquivo csv local todas as siglas dos papeis negociados na B3 e depois busca os dados na net

from selenium import webdriver
from time import sleep
import pandas as pd
!pip install yfinance --upgrade --no-cache-dir
!pip install python-telegram-bot
import time
import telegram
import yfinance as yf
import pandas as pd
yf.pdr_override()


#OBTER LISTA ATUALIZADA DOS PAPEIS
navegador = webdriver.Chrome()

navegador.get('https://sistemaswebb3-listados.b3.com.br/indexPage/day/IBOV?language=pt-br')
sleep(8)
navegador.find_element_by_link_text('Download').click()
navegador.close()

dadosBx = pd.read_csv("c://Users//johnn//Downloads//IBOVDia_19-05-21.csv",sep=';', encoding='ISO-8859-1', skipfooter=2, engine='python', thousands=',',header=1, index_col=False)

ticker = dadosBx['Código'] + '.SA'

pd.options.mode.chained_assignment = None



# ANALISE DE CADA PAPEL
for x in ticker:
    dia2 = yf.Ticker(x)
    Dia = dia2.history(period='1d', interval='5m').tail(1)
    Ano = yf.download(x, period='1y')
    print(x)     
    #ANO: criando coluna Data e transferindo valores
    Ano.insert(loc=0, column='DataMain', value=Ano.index)
    #remove a ultima data
    removedata = Ano.loc[(Ano['DataMain']==pd.to_datetime('today').normalize())]
    Ano = Ano.drop(removedata.index)
    #DIA: criando coluna Data e transferindo valores
    Dia.insert(loc=0, column='DataMain', value=Dia.index)
    #unindo os periodos
    Resumo = Ano.append(Dia)
    #ajuste da data
    Resumo['DataMain'] = pd.to_datetime(Resumo['DataMain'], utc=True).dt.date
    #resumindo as colunas desejadas (data e fechamento)
    ResumoEnd = Resumo[['DataMain', 'Close']]
    #calculo MACD
    rapidoMME = ResumoEnd.Close.ewm(span=12).mean()
    lentaMME = ResumoEnd.Close.ewm(span=26).mean()
    MACD = rapidoMME - lentaMME
    Sinal = MACD.ewm(span=9).mean()
    ResumoEnd['MACD'] = MACD
    ResumoEnd['Sinal'] = Sinal
    #ajuste index e retira data; coloca data como index
    ResumoEnd = ResumoEnd.set_index(pd.DatetimeIndex(ResumoEnd['DataMain'].values))
    ResumoEnd = ResumoEnd.drop('DataMain', 1)
    # criar código para verificar a compra ou venda
    ResumoEnd['flag'] = ''
    ResumoEnd['preco_compra'] = ''
    ResumoEnd['preco_venda'] = ''
    for i in range (1, len(ResumoEnd.Sinal)):
      if ResumoEnd['MACD'][i] > ResumoEnd['Sinal'][i]:
        if ResumoEnd['flag'][i-1] == 'C':
          ResumoEnd['flag'][i] = 'C'
        else:
          ResumoEnd['flag'][i] = 'C'
          ResumoEnd['preco_compra'][i] = ResumoEnd['Close'][i]
            
      elif ResumoEnd['MACD'][i] < ResumoEnd['Sinal'][i]:
        if ResumoEnd['flag'][i-1] == 'V':
            ResumoEnd['flag'][i] = 'V'
        else:
            ResumoEnd['flag'][i] = 'V'
            ResumoEnd['preco_venda'][i] = ResumoEnd['Close'][i]
            
    # mensagem ao TELEGRAM
    my_token = '1852343442:AAEBBS1NjjFRIqt-XTbb3rzRxipvk8ZqI5I'
    chat_id = '-351556985'
    def envia_mensagem(msg, chat_id, token = my_token):
      bot = telegram.Bot(token = token)
      bot.sendMessage(chat_id = chat_id, text=msg)
    hoje = ResumoEnd.flag[-1]
    ontem = ResumoEnd.flag[-2]
    flag = hoje
    ticker2 = x[0:-3]
    site = f'https://www.google.com/search?q={ticker2}&rlz=1C1EJFC_enBR915BR916&oq={ticker2}&aqs=chrome..69i57j0l5j0i10i433j69i60.4408j0j7&sourceid=chrome&ie=UTF-8'
    preco_fechamento = round(ResumoEnd.Close.tail(1)[-1],2)
    msg = f'{ticker2}  \n>>> {flag} <<< \nPreço de Fechamento: {preco_fechamento}\n\nTotal de ações analisadas: {len(ticker)}\n{site}'
    #verificar se houve alteração do sinal
    if ontem != hoje:
      envia_mensagem(msg, chat_id, my_token)
    #time.sleep(0.5)

[*********************100%***********************]  1 of 1 completed
ABEV3.SA
[*********************100%***********************]  1 of 1 completed
ASAI3.SA
[*********************100%***********************]  1 of 1 completed
AZUL4.SA
[*********************100%***********************]  1 of 1 completed
BTOW3.SA
[*********************100%***********************]  1 of 1 completed
B3SA3.SA
[*********************100%***********************]  1 of 1 completed
BIDI11.SA
[*********************100%***********************]  1 of 1 completed
BBSE3.SA
[*********************100%***********************]  1 of 1 completed
BRML3.SA
[*********************100%***********************]  1 of 1 completed
BBDC3.SA
[*********************100%***********************]  1 of 1 completed
BBDC4.SA
[*********************100%***********************]  1 of 1 completed
BRAP4.SA
[*********************100%***********************]  1 of 1 completed
BBAS3.SA
[*********************100%***********************]  1 of 1 comp

In [2]:
from selenium import webdriver
from time import sleep
import pandas as pd
!pip install yfinance --upgrade --no-cache-dir
!pip install python-telegram-bot
import time
import telegram
import yfinance as yf
import pandas as pd
yf.pdr_override()


#OBTER LISTA ATUALIZADA DOS PAPEIS
navegador = webdriver.Chrome()

navegador.get('https://sistemaswebb3-listados.b3.com.br/indexPage/day/IBOV?language=pt-br')
sleep(8)
navegador.find_element_by_link_text('Download').click()
navegador.close()

dadosBx = pd.read_csv("c://Users//johnn//Downloads//IBOVDia_19-05-21.csv",sep=';', encoding='ISO-8859-1', skipfooter=2, engine='python', thousands=',',header=1, index_col=False)

ticker = dadosBx['Código'] + '.SA'
ticker.to_csv('Atual.csv')